In [1]:
# General Imports

import numpy as np
import os
import time
from tqdm.notebook import tqdm

# Audio Imports

import librosa
from scipy.io import wavfile as wav
# import simpleaudio as sa
import IPython.display as ipd
import sounddevice as sd
# import wave

In [2]:
attempt = 0
name = "gian"
duration = 2
rec_rate = 22050 # sample rate of the provided recordings
recordings_dir = "my_recordings/"

In [3]:
if not os.path.isdir(recordings_dir):
    os.makedirs(recordings_dir)

In [4]:
i = 0
while i < 10:
    print(f"Recording attempt {attempt}")
    filename = recordings_dir+"_".join([str(attempt), name, str(i)])+".wav"
    print(f"\nNUMBER: {i}")
    print("Ready in 3...", end = "")
    time.sleep(1)
    print("2...", end = "")
    time.sleep(1)
    print("1...")
    time.sleep(1)
    print("Go.")
    rec = sd.rec(int(duration * rec_rate), samplerate=rec_rate, channels=1, blocking=True)
    print("Playing the recording.")
    sd.play(rec, rec_rate)
        
    # after hearing the recording, decide whether to record it again or continue to next number
    # if you type anything, record again
    # if you press enter, save current recording & go to next number
    ok = input("OK?")
    if ok == "":
        librosa.output.write_wav(filename, rec, rec_rate)
        i += 1
    ipd.clear_output(wait=True)
print("DONE")

DONE


# Remove silence from recordings

In [5]:
import subprocess

In [6]:
output_dir = "output/"
input_dir = "my_recordings/" # recordings_dir

In [7]:
# check if input directory exists
if not os.path.isdir(input_dir):
    print("There should be an input \"my recordings\" directory.")

# create output directory if not there yet
if not os.path.isdir(output_dir):
    os.makedirs(output_dir)

In [8]:
temp1 = output_dir+"temp1.wav"
temp2 = output_dir+"temp2.wav"
temp3 = output_dir+"temp3.wav"
db=-48 # default was -96, but it didn't cut anything; test with a few recordings before deciding
all_files = os.listdir(input_dir)

for file in tqdm(all_files):
    subprocess.run(["ffmpeg", "-y", "-i", input_dir+file, "-af", f"silenceremove=1:0:{db}dB", temp1])
    subprocess.run(["ffmpeg", "-y", "-i", temp1, "-af", "areverse", temp2])
    subprocess.run(["ffmpeg", "-y", "-i", temp2, "-af", f"silenceremove=1:0.1:{db}dB", temp3])
    subprocess.run(["ffmpeg", "-y", "-i", temp3, "-af", "areverse", output_dir+file])
    
    os.remove(temp1)
    os.remove(temp2)
    os.remove(temp3)

In [9]:
# check the durations of the cut recordings
# (shold be less than 2 seconds)
durations = []
for file in tqdm(os.listdir("output")):
    sample, sample_rate = librosa.core.load('output/'+file)
    duration = sample.shape[0]/sample_rate
    durations.append(duration)
durations

[0.5657596371882087,
 0.4443537414965986,
 0.2257142857142857,
 0.2531972789115646,
 0.3271201814058957,
 0.5336961451247165,
 0.4544671201814059,
 0.583718820861678,
 0.3543310657596372,
 0.5040816326530613]

In [10]:
# test if the cut makes sense (i.e., if number can still be heard)
for rec_file in tqdm(os.listdir("output")):
    rec_rate, rec = wav.read("output/"+rec_file)
    sd.play(rec, rec_rate)
    time.sleep(1) # without this, the audio cannot be heard...